# Labelling the data
First, I am going to labell the data using one of my previous projects - ImageInspector. You can install it by running the following command ```!pip install git+git://github.com/JanMalinowski/image_inspector.git```

In [1]:
from image_inspector import ImageInspector
import os
import pandas as pd

In [2]:
# Listing all the dobble files with dobble cards images
imgs = os.listdir('../input/images/')
path = '../input/images/'

In [3]:
# List of all the objects appearning on the cards (There're 57 of them, wow!)
categories = ['Anchor', 'Apple', 'Baby bottle', 'Bomb', 'Cactus','Candle', 'Taxi car', 'Carrot', 'Chess knight', 'Clock', 'Clown','Diasy flower', 'Dinosaur','Dolphin', 'Dragon', 'Exclamation point','Eye', 'Fire', 'Four leaf clover', 'Ghost', 'Green splats', 'Hammer','Heart', 'Ice cube', 'Igloo', 'Key', 'Ladybird (Ladybug)', 'Light bulb','Lightning bolt', 'Lock', 'Maple leaf', 'Moon', 'No Entry sign','Orange scarecrow man', 'Pencil', 'Purple bird', 'Purple cat','Purple dobble hand man', 'Red lips', 'Scissors', 'Skull and crossbones','Snowflake', 'Snowman', 'Spider', 'Spider’s web', 'Sun', 'Sunglasses','Target/crosshairs', 'Tortoise', 'Treble clef', 'Tree', 'Water drip','Dog', 'Yin and Yang', 'Zebra', 'Question mark', 'Cheese']

In [4]:
# Initializing the ImageInspector
ii = ImageInspector(imgs, path, categories, n_cols=5)

In [65]:
ii.iterator

218

In [5]:
# Labelling the data
ii()

AppLayout(children=(GridspecLayout(children=(ToggleButton(value=False, description='Anchor', layout=Layout(gri…

In [68]:
# Getting the dataframe with the results
results = ii.get_results()

In [69]:
results.head()

,Anchor,Apple,Baby bottle,Bomb,Cactus,Candle,Taxi car,Carrot,Chess knight,Clock,...,Tortoise,Treble clef,Tree,Water drip,Dog,Yin and Yang,Zebra,Question mark,Cheese,Image Name
0,False,False,False,False,False,False,False,False,False,False,...,False,True,True,False,False,True,True,False,False,IMG_20201022_013030.jpg
1,False,False,False,False,False,False,False,True,False,False,...,False,True,False,False,False,False,False,False,False,IMG_20201022_013434.jpg
2,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,IMG_20201022_011645.jpg
3,False,False,True,False,False,False,False,False,False,True,...,False,False,False,True,False,False,False,False,False,IMG_20201022_012859.jpg
4,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,True,True,False,False,True,IMG_20201022_014757.jpg


In [70]:
# Converting booleans to ints
results[categories] = results[categories].astype(int)
results.head()

,Anchor,Apple,Baby bottle,Bomb,Cactus,Candle,Taxi car,Carrot,Chess knight,Clock,...,Tortoise,Treble clef,Tree,Water drip,Dog,Yin and Yang,Zebra,Question mark,Cheese,Image Name
0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,1,1,0,0,IMG_20201022_013030.jpg
1,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,IMG_20201022_013434.jpg
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,IMG_20201022_011645.jpg
3,0,0,1,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,IMG_20201022_012859.jpg
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,1,0,0,1,IMG_20201022_014757.jpg


In [71]:
# Saving the dataframe to a pickle file.
results.to_pickle('../input/raw_data/data.pkl')

Having the pre-pre-trained neural network, we are going to train it to recognize elements, which are on two cards.
In order to do it, we need to create a dataset with pairs of images and the common object. The script ```create_dataset```, which does that requires us to give it a dataframe with two columns: images (containing images' names) and categories (containing list of objects appearing on a given card). We can create the dataframe using the code below.

In [72]:
cats = []
for index in results.index:
    temp = results.loc[index, categories].index[results.loc[index, categories]==1].tolist()
    cats.append(temp)

In [77]:
data_df = pd.DataFrame({'images': results['Image Name'], 'categories': cats})

In [12]:
# Creating a non overlapping train and test sets.
test_df = pd.DataFrame()

for i in range(15):
    temp = data_df.loc[data_df['categories'].apply(lambda x: x==data_df['categories'][i]),:]
    test_df = pd.concat([test_df, temp])
    
data_df = data_df.drop(test_df.index.values, axis=0).reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [13]:
test_df

,images,categories
0,IMG_20201022_011919.jpg,"[Exclamation point, Fire, Heart, Orange scarec..."
1,IMG_20201022_013410.jpg,"[Exclamation point, Fire, Heart, Orange scarec..."
2,IMG_20201022_013517.jpg,"[Exclamation point, Fire, Heart, Orange scarec..."
3,IMG_20201022_015003.jpg,"[Diasy flower, Ghost, Igloo, Skull and crossbo..."
4,IMG_20201022_012852.jpg,"[Diasy flower, Ghost, Igloo, Skull and crossbo..."
5,IMG_20201022_014001.jpg,"[Diasy flower, Ghost, Igloo, Skull and crossbo..."
6,IMG_20201022_011741.jpg,"[Diasy flower, Ghost, Igloo, Skull and crossbo..."
7,IMG_20201022_012130.jpg,"[Apple, Candle, Clock, Fire, Target/crosshairs..."
8,IMG_20201022_013224.jpg,"[Apple, Candle, Clock, Fire, Target/crosshairs..."
9,IMG_20201022_013650.jpg,"[Apple, Candle, Clock, Fire, Target/crosshairs..."


,images,categories
0,IMG_20201022_011919.jpg,"[Exclamation point, Fire, Heart, Orange scarec..."
1,IMG_20201022_015003.jpg,"[Diasy flower, Ghost, Igloo, Skull and crossbo..."
2,IMG_20201022_012130.jpg,"[Apple, Candle, Clock, Fire, Target/crosshairs..."
3,IMG_20201022_014644.jpg,"[Baby bottle, Cactus, Clown, Eye, Snowflake, S..."
4,IMG_20201022_013336.jpg,"[Chess knight, Clown, Diasy flower, Fire, Lady..."
...,...,...
165,IMG_20201022_014926.jpg,"[Taxi car, Clown, Four leaf clover, Maple leaf..."
166,IMG_20201022_014615.jpg,"[Fire, Ghost, Hammer, Ice cube, Key, Lightning..."
167,IMG_20201022_013233.jpg,"[Chess knight, Dragon, Igloo, Lightning bolt, ..."
168,IMG_20201022_013606.jpg,"[Candle, Carrot, Key, Moon, Snowman, Sun, Tort..."


In [78]:
# Saving the data for creating the dataset
data_df.to_pickle('../input/detecting_common_element/data.pkl')